In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [3]:
#Import thermodynamic data
therm = pd.read_excel('properties database.xlsx',sheetname='Thermwatprops',index_col='T')
therm.head()

,P,ro,SV,Cp,sH
T,,,,,
4,0.9,1000.0,1.0,4.205,NaN
5,0.9,1000.0,1.0,4.202,0.075
10,1.2,999.8,1.0,4.192,0.150
15,1.7,999.2,1.0,4.18551),0.223
20,2.3,998.3,1.0,4.182,0.296


In [4]:
#import input data for tower model
indat = pd.read_excel('data input.xlsx',sheetname='input',index_col='Variable')
indat

,Input,Units,Description
Variable,,,
RR,40000.000000,gpm,Recirc Rate
TR,93.000000,deg F,Return Temp
TRC,33.888889,deg C,Return Temp
TS,73.000000,deg F,Supply Temp
TSC,22.777778,deg C,Supply Temp
M0,NaN,kg/min,Makeup
M1,NaN,kg/min,Blowdown
M2,153846.153846,kg/min,Process Supply
M3,153846.153846,kg/min,Process Return


In [37]:
steam = pd.read_excel('properties database.xlsx',sheetname='Steamprops',index_col='T')
steam.head()

,P,SV,ro,HL,HV,HG,sHS
T,,,,,,,
3.8,0.8,160.00,0.00626,15.8,2493,2509,9.058
17.5,2.0,67.00,0.01490,73.5,2460,2534,8.725
32.9,5.0,28.20,0.03540,137.8,2424,2562,8.396
45.8,10.0,14.70,0.06820,191.8,2393,2585,8.151
60.1,20.0,7.65,0.13100,251.5,2358,2610,7.909


In [5]:
#Interpolation function

def interp(value,x,y):
    return np.interp(value,x,y)

In [69]:
#Create mass flow rate array
MCT = np.array([indat.loc['M0']['Input'],indat.loc['M1']['Input'],
                indat.loc['M2']['Input'],indat.loc['M3']['Input'],indat.loc['M4']['Input']])
#Create temperature array
T = np.array([indat.loc['T0']['Input'],indat.loc['T1']['Input'],indat.loc['T2']['Input'],
                indat.loc['T3']['Input'],indat.loc['T4']['Input']])
#Mass fraction array
COC = indat.loc['COC']['Input']
X = np.array([1,COC,COC,COC,0])

#Create array for specific enthalpy 
H = np.zeros(5)
for i in range(len(T)):
    H[i]=(interp(T[i],steam.index,steam['HL']))
H[4] = interp(T[4],steam.index,steam['HG'])
X

array([ 1.,  8.,  8.,  8.,  0.])

In [86]:
#Mass Balance equations
# M0+M3=M1+M2+M4
# M0*H0+M3*H3=M1*H1+M2*H2+M4*H4
# MO*X0+M3*X3=M1*X1+M2*X2+M4*X4
#Solver format a*X = b 
b1 = MCT[2]-MCT[3]
b2 = MCT[2]*H[2]-MCT[3]*H[3]
b3 = MCT[2]*X[2]-MCT[3]*X[3]
a1 = (1,-1,-1)
a2 = (H[0],-H[1],-H[4])
a3 = (X[0],-X[1],-X[4])

A = np.array([a1,a2,a3])
B = np.array([b1,b2,b3])

sol = np.linalg.lstsq(A,B)

MCT[0] = sol[0][0]
MCT[1] = sol[0][1]
MCT[4] = sol[0][2]

array([   863.14491233,    107.89311404,  40000.        ,  40000.        ,
          755.25179829])

In [ ]:
sol[3]